# Simulation with Keras library

In [1]:
from keras.models import Model 
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate

import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import dataLoader
from losses import dice_coefficient, dice_coefficient_loss

In [2]:
def conv_block(input, num_filters):
    x = Conv3D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv3D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPooling3D((2, 2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv3DTranspose(num_filters, (2, 2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
        activation = 'sigmoid'
    else:
        activation = 'softmax'

    outputs = Conv3D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [3]:
#Define parameters for our model.

patch_size = 64
channels=3

LR = 0.0001
optim = Adam(LR)

n_classes=4

In [4]:
model = build_unet((patch_size,patch_size,patch_size,channels), n_classes=n_classes)


softmax


In [8]:
#Generate the data
train_img, train_label = dataLoader.generate_data()

#splitting the train set 
X_train, X_test, y_train, y_test = train_test_split(train_img, train_label, test_size = 0.20, random_state = 0)

#zipping to train and val set 
#train_set = torch.cat((X_train, y_train), axis=1)
#val_set = torch.cat((X_test, y_test), axis=1)

NameError: name 'torch' is not defined

In [5]:
model.compile(optimizer = optim, loss=dice_coefficient_loss, metrics=dice_coefficient)
print(model.summary())

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 64,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 64, 64, 64,   5248        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 64,   256        ['conv3d[0][0]']                 
 alization)                     64)                                                           

                                                                                                  
 activation_7 (Activation)      (None, 8, 8, 8, 512  0           ['batch_normalization_7[0][0]']  
                                )                                                                 
                                                                                                  
 max_pooling3d_3 (MaxPooling3D)  (None, 4, 4, 4, 512  0          ['activation_7[0][0]']           
                                )                                                                 
                                                                                                  
 conv3d_8 (Conv3D)              (None, 4, 4, 4, 102  14156800    ['max_pooling3d_3[0][0]']        
                                4)                                                                
                                                                                                  
 batch_nor

 ormalization)                  128)                                                              
                                                                                                  
 activation_14 (Activation)     (None, 32, 32, 32,   0           ['batch_normalization_14[0][0]'] 
                                128)                                                              
                                                                                                  
 conv3d_15 (Conv3D)             (None, 32, 32, 32,   442496      ['activation_14[0][0]']          
                                128)                                                              
                                                                                                  
 batch_normalization_15 (BatchN  (None, 32, 32, 32,   512        ['conv3d_15[0][0]']              
 ormalization)                  128)                                                              
          

In [9]:
print(model.input_shape)
print(X_train.shape)
print(model.output_shape)
print(y_train.shape)
print("-------------------")

(None, 64, 64, 64, 3)
torch.Size([137, 9, 1024, 1024])
(None, 64, 64, 64, 4)
torch.Size([137, 9, 1024, 1024])
-------------------


In [10]:
#Fit the model
history=model.fit(X_train, 
          y_train,
          batch_size=8, 
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

TypeError: Cannot convert the argument `type_value`: torch.int16 to a TensorFlow DType.

In [ ]:
###
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['dice_coefficient']
val_acc = history.history['val_dice_coefficient']

plt.plot(epochs, acc, 'y', label='Training Dice')
plt.plot(epochs, val_acc, 'r', label='Validation Dice')
plt.title('Training and validation Dice')
plt.xlabel('Epochs')
plt.ylabel('Dice')
plt.legend()
plt.show()

In [ ]:
#Predict on the test data
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=1)
y_test_argmax = np.argmax(y_test, axis=1)